# 如何进行检索

:::info 预备知识

本指南假定您熟悉以下内容：

- [聊天机器人](/docs/tutorials/chatbot)
- [检索增强生成](/docs/tutorials/rag)

:::

检索是一种常见的技术，聊天机器人使用它来利用聊天模型训练数据之外的信息增强其回复。本节将介绍如何在聊天机器人中实现检索，但值得注意的是，检索是一个非常微妙且深奥的主题。

## 环境准备

您需要安装一些包，并设置所需的LLM API密钥：

```{=mdx}
import Npm2Yarn from "@theme/Npm2Yarn";

<Npm2Yarn>
  @langchain/openai @langchain/core cheerio
</Npm2Yarn>
```

接下来，我们还需要设置一个聊天模型，用于以下示例。

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```

In [ ]:
// @lc-docs-hide-cell

import { ChatOpenAI } from "@langchain/openai";

const llm = new ChatOpenAI({
  model: "gpt-4o",
  temperature: 0,
});

## 创建一个检索器

我们将使用[LangSmith 文档](https://docs.smith.langchain.com)作为源材料，并将内容存储在向量数据库中以供后续检索。请注意，此示例将略过有关解析和存储数据源的一些具体细节——你可以[在此处查看有关创建检索系统的深入文档](/docs/how_to/#qa-with-rag)。

让我们使用文档加载器从文档中提取文本：

In [1]:
import "cheerio";
import { CheerioWebBaseLoader } from "@langchain/community/document_loaders/web/cheerio";

const loader = new CheerioWebBaseLoader(
  "https://docs.smith.langchain.com/user_guide"
);

const rawDocs = await loader.load();

rawDocs[0].pageContent.length;

36687

接下来，我们将其拆分为较小的块，以便LLM的上下文窗口可以处理，并将其存储在向量数据库中：

In [2]:
import { RecursiveCharacterTextSplitter } from "langchain/text_splitter";

const textSplitter = new RecursiveCharacterTextSplitter({
  chunkSize: 500,
  chunkOverlap: 0,
});

const allSplits = await textSplitter.splitDocuments(rawDocs);

然后我们将这些块嵌入并存储在向量数据库中：

In [3]:
import { OpenAIEmbeddings } from "@langchain/openai";
import { MemoryVectorStore } from "langchain/vectorstores/memory";

const vectorstore = await MemoryVectorStore.fromDocuments(
  allSplits,
  new OpenAIEmbeddings()
);

最后，让我们从初始化的向量存储中创建一个检索器：

In [4]:
const retriever = vectorstore.asRetriever(4);

const docs = await retriever.invoke("how can langsmith help with testing?");

console.log(docs);

[
  Document {
    pageContent: "These test cases can be uploaded in bulk, created on the fly, or exported from application traces. L"... 294 more characters,
    metadata: {
      source: "https://docs.smith.langchain.com/user_guide",
      loc: { lines: { from: 7, to: 7 } }
    }
  },
  Document {
    pageContent: "We provide native rendering of chat messages, functions, and retrieve documents.Initial Test Set​Whi"... 347 more characters,
    metadata: {
      source: "https://docs.smith.langchain.com/user_guide",
      loc: { lines: { from: 6, to: 6 } }
    }
  },
  Document {
    pageContent: "will help in curation of test cases that can help track regressions/improvements and development of "... 393 more characters,
    metadata: {
      source: "https://docs.smith.langchain.com/user_guide",
      loc: { lines: { from: 11, to: 11 } }
    }
  },
  Document {
    pageContent: "that time period — this is especially handy for debugging production issues.LangSmith also allows fo"... 39

我们可以看到，调用上面的检索器会返回一些LangSmith文档的部分内容，这些内容包含有关测试的信息，我们的聊天机器人可以在回答问题时用作上下文。现在我们有了一个可以从LangSmith文档中返回相关数据的检索器！

## 文档链

既然我们已经有了一个可以返回LangChain文档的检索器，那么我们现在来创建一个链，使其能够使用这些文档作为上下文来回答问题。我们将使用一个`createStuffDocumentsChain`辅助函数，将所有输入文档“填充”到提示词中。它还将负责将文档格式化为字符串。

除了聊天模型外，该函数还需要提供一个包含`context`变量的提示词，以及一个名为`messages`的聊天历史消息占位符。我们将创建一个合适的提示词并按如下方式传递：

In [5]:
import { createStuffDocumentsChain } from "langchain/chains/combine_documents";
import {
  ChatPromptTemplate,
  MessagesPlaceholder,
} from "@langchain/core/prompts";

const SYSTEM_TEMPLATE = `Answer the user's questions based on the below context. 
If the context doesn't contain any relevant information to the question, don't make something up and just say "I don't know":

<context>
{context}
</context>
`;

const questionAnsweringPrompt = ChatPromptTemplate.fromMessages([
  ["system", SYSTEM_TEMPLATE],
  new MessagesPlaceholder("messages"),
]);

const documentChain = await createStuffDocumentsChain({
  llm,
  prompt: questionAnsweringPrompt,
});

我们可以单独调用此 `documentChain` 来回答问题。让我们使用上面检索到的文档和相同的问题 `langsmith 如何帮助测试？`:

In [6]:
import { HumanMessage, AIMessage } from "@langchain/core/messages";

await documentChain.invoke({
  messages: [new HumanMessage("Can LangSmith help test my LLM applications?")],
  context: docs,
});

"Yes, LangSmith can help test your LLM applications. It allows developers to create datasets, which a"... 229 more characters

看起来不错！作为对比，我们可以尝试不使用上下文文档并比较结果：

In [7]:
await documentChain.invoke({
  messages: [new HumanMessage("Can LangSmith help test my LLM applications?")],
  context: [],
});

"I don't know."

我们可以看到，LLM 未返回任何结果。

## 检索链

让我们将此文档链与检索器结合。以下是一种可能的实现方式：

In [8]:
import type { BaseMessage } from "@langchain/core/messages";
import {
  RunnablePassthrough,
  RunnableSequence,
} from "@langchain/core/runnables";

const parseRetrieverInput = (params: { messages: BaseMessage[] }) => {
  return params.messages[params.messages.length - 1].content;
};

const retrievalChain = RunnablePassthrough.assign({
  context: RunnableSequence.from([parseRetrieverInput, retriever]),
}).assign({
  answer: documentChain,
});

给定一个输入消息列表，我们提取列表中最后一条消息的内容，并将其传递给检索器以获取一些文档。然后，我们将这些文档作为上下文传递给我们的文档链以生成最终响应。
调用此链将合并上述两个步骤：

In [9]:
await retrievalChain.invoke({
  messages: [new HumanMessage("Can LangSmith help test my LLM applications?")],
});

{
  messages: [
    HumanMessage {
      lc_serializable: true,
      lc_kwargs: {
        content: "Can LangSmith help test my LLM applications?",
        additional_kwargs: {},
        response_metadata: {}
      },
      lc_namespace: [ "langchain_core", "messages" ],
      content: "Can LangSmith help test my LLM applications?",
      name: undefined,
      additional_kwargs: {},
      response_metadata: {}
    }
  ],
  context: [
    Document {
      pageContent: "These test cases can be uploaded in bulk, created on the fly, or exported from application traces. L"... 294 more characters,
      metadata: {
        source: "https://docs.smith.langchain.com/user_guide",
        loc: { lines: [Object] }
      }
    },
    Document {
      pageContent: "this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each s"... 343 more characters,
      metadata: {
        source: "https://docs.smith.langchain.com/user_guide",
        loc: { lines: [Object

看起来不错！

## 查询转换

我们的检索链能够回答有关 LangSmith 的问题，但存在一个问题——聊天机器人以对话方式与用户交互，因此必须处理后续问题。

当前形式的链将难以应对这种情况。例如，对于我们最初的问题的一个后续问题 `告诉我更多！`。如果我们直接使用该查询调用检索器，我们将得到与 LLM 应用测试无关的文档：

In [10]:
await retriever.invoke("Tell me more!");

[
  Document {
    pageContent: "Oftentimes, changes in the prompt, retrieval strategy, or model choice can have huge implications in"... 40 more characters,
    metadata: {
      source: "https://docs.smith.langchain.com/user_guide",
      loc: { lines: { from: 8, to: 8 } }
    }
  },
  Document {
    pageContent: "This allows you to quickly test out different prompts and models. You can open the playground from a"... 37 more characters,
    metadata: {
      source: "https://docs.smith.langchain.com/user_guide",
      loc: { lines: { from: 10, to: 10 } }
    }
  },
  Document {
    pageContent: "We provide native rendering of chat messages, functions, and retrieve documents.Initial Test Set​Whi"... 347 more characters,
    metadata: {
      source: "https://docs.smith.langchain.com/user_guide",
      loc: { lines: { from: 6, to: 6 } }
    }
  },
  Document {
    pageContent: "together, making it easier to track the performance of and annotate your application across multiple"... 244 

这是因为检索器没有内在的状态概念，只会提取与给定查询最相似的文档。为了解决这个问题，我们可以利用LLM将查询转换为一个不包含任何外部引用的独立查询。

以下是一个示例：

In [11]:
const queryTransformPrompt = ChatPromptTemplate.fromMessages([
  new MessagesPlaceholder("messages"),
  [
    "user",
    "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation. Only respond with the query, nothing else.",
  ],
]);

const queryTransformationChain = queryTransformPrompt.pipe(llm);

await queryTransformationChain.invoke({
  messages: [
    new HumanMessage("Can LangSmith help test my LLM applications?"),
    new AIMessage(
      "Yes, LangSmith can help test and evaluate your LLM applications. It allows you to quickly edit examples and add them to datasets to expand the surface area of your evaluation sets or to fine-tune a model for improved quality or reduced costs. Additionally, LangSmith can be used to monitor your application, log all traces, visualize latency and token usage statistics, and troubleshoot specific issues as they arise."
    ),
    new HumanMessage("Tell me more!"),
  ],
});

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: '"LangSmith LLM application testing and evaluation features"',
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: '"LangSmith LLM application testing and evaluation features"',
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 11, promptTokens: 144, totalTokens: 155 },
    finish_reason: "stop"
  },
  tool_calls: [],
  invalid_tool_calls: []
}

太棒了！这个转换后的查询将提取与LLM应用测试相关的上下文文档。

让我们将其添加到检索链中。我们可以按如下方式封装检索器：

In [12]:
import { RunnableBranch } from "@langchain/core/runnables";
import { StringOutputParser } from "@langchain/core/output_parsers";

const queryTransformingRetrieverChain = RunnableBranch.from([
  [
    (params: { messages: BaseMessage[] }) => params.messages.length === 1,
    RunnableSequence.from([parseRetrieverInput, retriever]),
  ],
  queryTransformPrompt
    .pipe(llm)
    .pipe(new StringOutputParser())
    .pipe(retriever),
]).withConfig({ runName: "chat_retriever_chain" });

然后，我们可以使用此查询转换链来改进检索链，使其更能处理此类后续问题：


In [13]:
const conversationalRetrievalChain = RunnablePassthrough.assign({
  context: queryTransformingRetrieverChain,
}).assign({
  answer: documentChain,
});

太棒了！让我们用与之前相同的输入来调用这个新链：


In [14]:
await conversationalRetrievalChain.invoke({
  messages: [new HumanMessage("Can LangSmith help test my LLM applications?")],
});

{
  messages: [
    HumanMessage {
      lc_serializable: true,
      lc_kwargs: {
        content: "Can LangSmith help test my LLM applications?",
        additional_kwargs: {},
        response_metadata: {}
      },
      lc_namespace: [ "langchain_core", "messages" ],
      content: "Can LangSmith help test my LLM applications?",
      name: undefined,
      additional_kwargs: {},
      response_metadata: {}
    }
  ],
  context: [
    Document {
      pageContent: "These test cases can be uploaded in bulk, created on the fly, or exported from application traces. L"... 294 more characters,
      metadata: {
        source: "https://docs.smith.langchain.com/user_guide",
        loc: { lines: [Object] }
      }
    },
    Document {
      pageContent: "this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each s"... 343 more characters,
      metadata: {
        source: "https://docs.smith.langchain.com/user_guide",
        loc: { lines: [Object

In [19]:
await conversationalRetrievalChain.invoke({
  messages: [
    new HumanMessage("Can LangSmith help test my LLM applications?"),
    new AIMessage(
      "Yes, LangSmith can help test and evaluate your LLM applications. It allows you to quickly edit examples and add them to datasets to expand the surface area of your evaluation sets or to fine-tune a model for improved quality or reduced costs. Additionally, LangSmith can be used to monitor your application, log all traces, visualize latency and token usage statistics, and troubleshoot specific issues as they arise."
    ),
    new HumanMessage("Tell me more!"),
  ],
});

{
  messages: [
    HumanMessage {
      lc_serializable: true,
      lc_kwargs: {
        content: "Can LangSmith help test my LLM applications?",
        additional_kwargs: {},
        response_metadata: {}
      },
      lc_namespace: [ "langchain_core", "messages" ],
      content: "Can LangSmith help test my LLM applications?",
      name: undefined,
      additional_kwargs: {},
      response_metadata: {}
    },
    AIMessage {
      lc_serializable: true,
      lc_kwargs: {
        content: "Yes, LangSmith can help test and evaluate your LLM applications. It allows you to quickly edit examp"... 317 more characters,
        tool_calls: [],
        invalid_tool_calls: [],
        additional_kwargs: {},
        response_metadata: {}
      },
      lc_namespace: [ "langchain_core", "messages" ],
      content: "Yes, LangSmith can help test and evaluate your LLM applications. It allows you to quickly edit examp"... 317 more characters,
      name: undefined,
      additional_kwargs: 

你可以查看 [这个 LangSmith 跟踪](https://smith.langchain.com/public/dc4d6bd4-fea5-45df-be94-06ad18882ae9/r)，亲自了解内部查询转换步骤。

## 流式传输

由于此链是使用 LCEL 构建的，因此你可以使用熟悉的 `.stream()` 方法：

In [15]:
const stream = await conversationalRetrievalChain.stream({
  messages: [
    new HumanMessage("Can LangSmith help test my LLM applications?"),
    new AIMessage(
      "Yes, LangSmith can help test and evaluate your LLM applications. It allows you to quickly edit examples and add them to datasets to expand the surface area of your evaluation sets or to fine-tune a model for improved quality or reduced costs. Additionally, LangSmith can be used to monitor your application, log all traces, visualize latency and token usage statistics, and troubleshoot specific issues as they arise."
    ),
    new HumanMessage("Tell me more!"),
  ],
});

for await (const chunk of stream) {
  console.log(chunk);
}

{
  messages: [
    HumanMessage {
      lc_serializable: true,
      lc_kwargs: {
        content: "Can LangSmith help test my LLM applications?",
        additional_kwargs: {},
        response_metadata: {}
      },
      lc_namespace: [ "langchain_core", "messages" ],
      content: "Can LangSmith help test my LLM applications?",
      name: undefined,
      additional_kwargs: {},
      response_metadata: {}
    },
    AIMessage {
      lc_serializable: true,
      lc_kwargs: {
        content: "Yes, LangSmith can help test and evaluate your LLM applications. It allows you to quickly edit examp"... 317 more characters,
        tool_calls: [],
        invalid_tool_calls: [],
        additional_kwargs: {},
        response_metadata: {}
      },
      lc_namespace: [ "langchain_core", "messages" ],
      content: "Yes, LangSmith can help test and evaluate your LLM applications. It allows you to quickly edit examp"... 317 more characters,
      name: undefined,
      additional_kwargs: 

## 下一步
你现在已经学习了一些将个人数据作为上下文添加到聊天机器人中的技术。

本指南仅涉及检索技术的表面内容。如需了解更多关于数据摄入、准备和检索最相关数据的不同方法，请查看我们的[检索指南](/docs/how_to/#retrievers)。